# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **ML Model - Predict Remaining Useful Life (RUL)**

## Objectives

Answer [Business Requirement 1](https://github.com/roeszler/filter-maintenance-predictor/blob/main/README.md#business-requirements) :
*   Fit and evaluate a **regression model** to predict the Remaining Useful Life of a replaceable part
*   Fit and evaluate a **classification model** to predict the Remaining Useful Life of a replaceable part should the regressor not perform well.

## Inputs

Data cleaning:
* outputs/datasets/cleaned/dfCleanTotal.csv

## Outputs

* Train set (features and target)
* Test set (features and target)
* Validation set (features and target)
* ML pipeline to predict RUL
* A map of the labels
* Feature Importance Plot



---

### Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

In [ ]:
current_dir = os.getcwd()
current_dir

---

## The major steps in this Regressor Pipeline

1. **ML Pipeline: Regressor**
    * Create Regressor Pipeline
    * Split the train set
    * Grid Search CV SKLearn
        * Use standard hyperparameters to find most suitable algorithm
        * Extensive search on most suitable algorithm to find the best hyperparameter configuration
    * Assess Feature Performance
    * Evaluate Regressor
    * Create Train, Test, Validation Sets

2. **ML Pipeline: Regressor + Principal Component Analysis (PCA)**
    * Prepare the Data for the Pipeline
    * Create Regressor + PCA Pipeline
    * Split the train and validation sets
    * Grid Search CV SKLearn
        * Use standard hyperparameters to find most suitable algorithm
        * Do an extensive search on most suitable algorithm to find the best hyperparameter configuration
    * Assess Feature Performance
    * Evaluate Regressor
    * Create Train, Test, Validation Sets

_Optionally_

3. **Convert Regression to Classification**
    * Convert numerical target to bins, and check if it is balanced
    * Rewrite Pipeline for ML Modelling
    * Load Algorithms For Classification
    * Split the Train Test sets:
    * Grid Search CV SKLearn:
        * Use standard hyper parameters to find most suitable model
        * Grid Search CV
        * Check Result
    * Do an extensive search on the most suitable model to find the best hyperparameter configuration.
        * Define Model Parameters
        * Extensive Grid Search CV                             
        * Check Results
        * Check Best Model
        * Parameters for best model
        * Define the best clf_pipeline
    * Assess Feature Importance
    * Evaluate Classifier on Train and Test Sets
        * Custom Function
        * List that relates the classes and tenure interval

4. **Decide which pipeline to use**

5. **Refit with the best features**
    * Rewrite Pipeline
    * Split Train Test Set with only best features
    * Subset best features
    * Grid Search CV SKLearn
    * Best Parameters
        * Manually
    * Grid Search CV
    * Check Results
    * Check Best Model
    * Define the best pipeline

6. **Assess Feature Importance**

7. **Push Files to Repo**

<!-- Modelling:
The hypothesis part of the process where you will find out whether you can answer the question.
* Identify what techniques to use.
* Split your data into train, validate and test sets.
* Build and train the models with the train data set.
* Validate Models and hyper-parameter : Trial different machine learning methods and models with the validation data set.
* Poor Results - return to data preparation for feature engineering
* Successful hypothesis - where the inputs from the data set are mapped to the output target / label appropriately to evaluate.

5. Evaluation:
Where you test whether the model can predict unseen data.
* Test Dataset
* Choose the model that meets the business success criteria best.
* Review and document the work that you have done.
* If your project meets the success metrics you defined with your customer?
- Ready to deploy. -->

---

### Load Cleaned Data
The pipeline should handle the cleaning and engineering by itself

In [ ]:
# import numpy as np
# import pandas as pd
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
# ! chmod 600 kaggle.json

# KaggleDatasetPath = 'prognosticshse/preventive-to-predicitve-maintenance'
# DestinationFolder = 'inputs/datasets/raw'   
# ! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

# ! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
#   && rm {DestinationFolder}/*.zip \
#   && rm {DestinationFolder}/*.pdf \
#   && rm {DestinationFolder}/*.mat \
# #   && rm kaggle.json

# df_test = pd.read_csv(f'inputs/datasets/raw/Test_Data_CSV.csv')
# df_train = pd.read_csv(f'inputs/datasets/raw/Train_Data_CSV.csv')
# # df_total = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTotal.csv')
# df_total = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
import numpy as np
import pandas as pd
df_total = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTotal.csv')
# df_total = pd.read_csv(f'outputs/datasets/transformed/dfTransformedTotal.csv') # data with negative log_EWM values removed
# df_total = (pd.read_csv("outputs/datasets/collection/PredictiveMaintenanceTotal.csv").drop(labels=['customerID', 'TotalCharges', 'Churn'], axis=1))
print(df_total.shape)
# df_total

### Include 4 Point Exponentially Weighted Mean and Remove Negative Values

**Note:** A runtime warning indicating a divide by zero is expected in the calculation of the log_EWM. We are using this to identify which values to delete, so have temporarily suppressed the warning for this calculation.

In [ ]:
# Create Log EWM to identify values to remove
import warnings
warnings.filterwarnings('ignore')

df_means = pd.DataFrame()

list_data_nos = list(df_total['Data_No'].unique())
for n in list_data_nos:
    if (df_total.Data_No != df_total.Data_No.shift(1)).any().any():
        df_bin = df_total[df_total['Data_No'] == n]

        ewm_calc = df_bin['Differential_pressure'].ewm(span=4, adjust=False).mean()
        df_bin.insert(loc=2, column='4point_EWM', value=ewm_calc)

        log_ewm = np.log(ewm_calc)
        df_bin.insert(loc=3, column='log_EWM', value=log_ewm)

        df_means = pd.concat([df_means, df_bin], ignore_index = True)
df_total = df_means

warnings.resetwarnings()

# Delete Negatives
data = df_total.loc[:, df_total.columns == 'log_EWM']
df_total = df_total[data.select_dtypes(include=[np.number]).ge(-0).all(1)].reset_index(drop=True)

# Delete engineered values
del df_total['log_EWM']
# del df_total['4point_EWM']

# df_total.loc[391:397]
# print(df_total.shape)
df_total

# ML Pipeline : Regressor
## Create Regressor Pipeline
### Set the Transformations
* Smart correlation
* feat_scaling
* feat_selection
* Modelling
* Model as variable

Note: Numerical Transformation not required as data supplied as integers

#### ML Pipeline for **Fitting Models** (regression)
Modelling and Hyperparameter Optimization

In [ ]:
# def PipelineOptimization(model):
#     pipeline_base = Pipeline([
#         # ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
#         #                                              variables=['gender', 'Partner', 'Dependents', 'PhoneService',
#         #                                                         'MultipleLines', 'InternetService', 'OnlineSecurity',
#         #                                                         'OnlineBackup', 'DeviceProtection', 'TechSupport',
#         #                                                         'StreamingTV', 'StreamingMovies', 'Contract',
#         #                                                         'PaperlessBilling', 'PaymentMethod'])),
#         ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")),
#         ("feat_scaling", StandardScaler()),
#         ("feat_selection",  SelectFromModel(model)),
#         ("model", model),
#     ])
#     return pipeline_base

In [ ]:
# Feature Management
from sklearn.pipeline import Pipeline
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.feature_selection import SelectFromModel

# ML regression algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression

# # ML classification algorithms
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
# from xgboost import XGBClassifier

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Data_No', 'Differential_pressure', '4point_EWM', 'Flow_rate',
                                                                'Time', 'Dust_feed', 'Dust', 'RUL', 'mass_g',
                                                                'cumulative_mass_g', 'Tt', 'filter_balance'])),
        ('SmartCorrelatedSelection', SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")),
        ('feat_scaling', StandardScaler()),
        ('feat_selection', SelectFromModel(model)),
        ('model', model),])
    return pipeline_base

In [ ]:
# model = PipelineOptimization(self.models[key])

#### **Custom Class** to fit a set of algorithms, each with its own set of hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            model = PipelineOptimization(self.models[key])
            print(f"\nRunning GridSearchCV for {key} \n")

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

Convert `Dust` to floating variable

In [ ]:
dust_density_total = [0.900 if n == 'ISO 12103-1, A2 Fine Test Dust' else (1.025 if n == 'ISO 12103-1, A3 Medium Test Dust' else 1.200) for n in df_total['Dust']]
df_total['Dust'] = dust_density_total

Add Filter Balance to **df_total**

In [ ]:
df_total_dp = df_total['Differential_pressure']
df_test_filter_balance = (((600 - df_total_dp)/600)*100).round(decimals = 2)
df_total.insert(loc=8, column='filter_balance', value=df_test_filter_balance)

## Split the data into Train, Test, Validate

Data is discrete however in bins, so:
#### Extract Cleaned **Train** & **Test** Datasets

In [ ]:
n = df_total['Data_No'].iloc[0:len(df_total)]
df_train = df_total[n < 51].reset_index(drop=True)
df_test = df_total[n > 50].reset_index(drop=True)
del df_train['RUL']
df_train

In [ ]:
df_test

#### Engineer Even Distribution of `Dust` in **Train** dataset

In [ ]:
df_train_copy = df_train.copy()

bin_sum = df_train_copy.groupby('Data_No')['Data_No'].count().reset_index(name='bin_tot')
map_bin = df_train_copy['Data_No'].map(bin_sum.set_index('Data_No')['bin_tot'])
df_train_copy.loc[:, 'bin_size'] = map_bin

dust_A2 = df_train_copy[df_train_copy['Dust'] == 0.900]
filter_A2 = dust_A2[dust_A2.Data_No != dust_A2.Data_No.shift(-1)]
df_train_A2 = filter_A2.sort_values(by='filter_balance', ascending=True)
df_train_A2['c_sum'] = df_train_A2['bin_size'].cumsum()
# df_train_A2.head(13).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

dust_A3 = df_train_copy[df_train_copy['Dust'] == 1.025]
filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3 = filter_A3.sort_values(by='filter_balance', ascending=True)
df_train_A3['c_sum'] = df_train_A3['bin_size'].cumsum()
# dn_fb = df_train_A3.loc[:, 'Data_No'].head(14).sort_values(ascending=True).reset_index(drop=True)
# df_train_A3.head(14).style.hide(['Time', 'Dust_feed', 'Flow_rate', 'Dust', 'mass_g', 'cumulative_mass_g', 'Tt'], axis="columns")

A2_bin = df_train_A2['Data_No'].head(9)
A3_bin = df_train_A3['Data_No'].head(9)

df_train_cleaned_A1 = df_train_copy[df_train_copy['Dust'] == 1.200]
df_train_cleaned_A2 = df_train_copy[df_train_copy['Data_No'].isin(A2_bin)]
df_train_cleaned_A3 = df_train_copy[df_train_copy['Data_No'].isin(A3_bin)]

df_train_concat = pd.concat([df_train_cleaned_A1, df_train_cleaned_A2, df_train_cleaned_A3], ignore_index = True)
df_train_model = df_train_concat.sort_values(by='Data_No', ascending=True)

print('A1 Dust_train :', df_train_cleaned_A1.shape)
print('A2 Dust_train :', df_train_cleaned_A2.shape)
print('A3 Dust_train :', df_train_cleaned_A3.shape)
print('\nTotal df_train :', df_train_model.shape)

In [ ]:
df_train_model

#### Determine **Target** and **Independent** Variables and Extract **Validation** Dataset

As discussed in the readme, this data has been supplied pre-split into **train** and **test** within unique **data bins**. 
We extract random observations from the **test** dataset to create a **validation** set, in a 70:30 split.


In [ ]:
df_test

In [ ]:
# from feature_engine.selection import SmartCorrelatedSelection
# corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")

# df_engineering = df_test.copy()
# corr_sel.fit_transform(df_engineering)
# print('Correlated Variables :\n', corr_sel.correlated_feature_sets_)
# print('\nFeatures to Drop :\n', corr_sel.features_to_drop_)

Review correlations, Drop Features and Split into **70% test** and **30% validate**. 

In [ ]:
from sklearn.model_selection import train_test_split
from feature_engine.selection import SmartCorrelatedSelection

corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")
df_engineering = df_test.copy()
corr_sel.fit_transform(df_engineering)

X = df_test.drop(corr_sel.features_to_drop_,axis=1)
y = df_test['Differential_pressure']

X_test, X_validate, y_test, y_validate = train_test_split(X,y,test_size=0.30, random_state=0)

print(X_test.shape, 'X_test')
print(X_validate.shape, 'X_validate')
print(y_test.shape, 'y_test')
print(y_validate.shape, 'y_validate')
print('\nFeatures Dropped :\n', corr_sel.features_to_drop_)

In [ ]:
X_test

#### Define **X_train**, **y_train** variables

In [ ]:
df_train

In [ ]:
corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6, selection_method="variance")
df_engineering = df_train.copy()
corr_sel.fit_transform(df_engineering)

X_train = df_train.drop(corr_sel.features_to_drop_,axis=1)
y_train = df_train['Differential_pressure']

print(X_train.shape, 'X_train')
print(y_train.shape, 'y_train')
print('\nFeatures Dropped :\n', corr_sel.features_to_drop_)

In [ ]:
X_train

In [ ]:
y_train

## Handling Target Imbalance
### No need to handle target imbalance in this **regression model**.
* Typically we only need to create a single pipeline for Classification or Regression task. 
* The exception occurs when we need to handle a **classification target imbalance**, which requires more than one model to process 

## Fit the pipeline with Data
* Prepare the data for handling the train set and
* ~~Fix the target imbalance~~
* Feature scaling
* Feature selection
* Modelling
* Custom Python class for **hyperparameter optimization**

### Use standard hyperparameters to find most suitable algorithm for the data

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeRegressor': DecisionTreeRegressor(random_state=0),
    'RandomForestRegressor': RandomForestRegressor(random_state=0),
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=0),
    'AdaBoostRegressor': AdaBoostRegressor(random_state=0),
    'GradientBoostingRegressor': GradientBoostingRegressor(random_state=0),
    'XGBRegressor': XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    'LogisticRegression': {},
    'DecisionTreeRegressor': {},
    'RandomForestRegressor': {},
    'ExtraTreesRegressor': {},
    'AdaBoostRegressor': {},
    'GradientBoostingRegressor': {},
    'XGBRegressor': {},
}

#### Fit the pipelines, using the above models with **default hyperparameters**
* Parsed the train set
* Set the performance metric as an R2 score (Regression: described in our ML business case)
* Cross validation as 5 (rule of thumb)

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

#### Observations

* The average R2 score is around **0.46**, which is lower than the **0.7** we decided in the business case.
* The best result is **GradientBoostingRegressor**.
* Perform an extensive search to hopefully improve performance.

In [ ]:
df_stop

### Train the Model

Multiple regression and classification models under consideration 

* sklearn.linear_model.**LinearRegression**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
* sklearn.linear_model.**LogisticRegression**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
    * *.predict_proba(X)*
* sklearn.linear_model.**SGDRegressor**(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
    * *.SGDClassifier()*

List full of available and under consideration can be seen at scikitlearn [linear models](https://scikit-learn.org/stable/modules/linear_model.html#)

* No one optimal model. the most appropriate seems .LogisticRegression()
<!-- 
**.LinearRegression()** - Ordinary Least Squares
**.SGDClassifier()** and **.SGDRegressor()** - Stochastic Gradient Descent - SGD
.Ridge() 
.Lasso()
.MultiTaskLasso()
.ElasticNet()
.MultiTaskElasticNet()
.Lars() - Least Angle Regression
.LassoLars()
.OrthogonalMatchingPursuit() and orthogonal_mp()
.BayesianRidge() - Bayesian Regression
.ARDRegression() - Automatic Relevance Determination
Generalized Linear Models
**.LogisticRegression()** + **.predict_proba(X)**
.TweedieRegressor()
.Perceptron()
.PassiveAggressiveClassifier() and .PassiveAggressiveRegressor()
Robustness regression: outliers and modeling errors
.RANSACRegressor()
.TheilSenRegressor() and 
.HuberRegressor()
.QuantileRegressor()
Polynomial regression: extending linear models with basis functions
.PolynomialFeatures() transformer -->


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)

In [ ]:
from sklearn.linear_model import SGDRegressor
SGDreg = SGDRegressor()
SGDreg.fit(X_train,y_train)

### Predictions and Model Evaluation

In [ ]:
from sklearn.metrics import classification_report

prediction = logrig.predict(X_test)
print(classification_report(y_test,prediction))